### **HuggingFace Login**

In [1]:
from huggingface_hub import login
login()

### **Import Libairies**

In [2]:
from IPython.display import display

import numpy as np
import pandas as pd
from tqdm import tqdm
import torch

from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
from sklearn.model_selection import train_test_split

from transformers import Trainer, TrainingArguments
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from datasets import Dataset



### **Read Datasets**

In [3]:
'''
df_train = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv", index_col=0)
df_test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv", index_col=0)

df_train = pd.read_csv("/media/yanncauchepin/ExternalDisk/Datasets/NaturalLanguageProcessing/disaster_tweets/train.csv", index_col=0)
df_test = pd.read_csv("/media/yanncauchepin/ExternalDisk/Datasets/NaturalLanguageProcessing/disaster_tweets/test.csv", index_col=0)
'''

df_train = pd.read_csv("C:/Users/cauchepy/Datasets/NaturalLanguageProcessing/kaggle_disastertweets/train.csv", index_col=0)
df_test = pd.read_csv("C:/Users/cauchepy/Datasets/NaturalLanguageProcessing/kaggle_disastertweets/test.csv", index_col=0)


### **Short Analysis**

In [4]:
print(f"Length - train {len(df_train)} - test {len(df_test)}")

Length - train 7613 - test 3263


### **Preprocess Datasets**

##### _Merge columns (full)_

In [5]:
keywords = pd.concat([
    pd.DataFrame(df_train["keyword"].value_counts()).rename(columns={"count":"train"}),
    pd.DataFrame(df_test["keyword"].value_counts()).rename(columns={"count":"test"})
    ],
    axis=1
)
keywords.head()

,train,test
keyword,,
fatalities,45,5
deluge,42,8
armageddon,42,8
sinking,41,9
damage,41,9


In [6]:
locations = pd.concat([
    pd.DataFrame(df_train["location"].value_counts()).rename(columns={"count":"train"}),
    pd.DataFrame(df_test["location"].value_counts()).rename(columns={"count":"test"})
    ],
    axis=1
)
locations.head()

,train,test
location,,
USA,104.0,37.0
New York,71.0,38.0
United States,50.0,15.0
London,45.0,13.0
Canada,29.0,13.0


In [7]:
df_train_full = df_train.copy()
df_train_full['text'] = df_train_full.apply(lambda row: f"{row['location'] or ''} {row['keyword'] or ''} {row['text']}".strip(), axis=1)
df_test_full = df_test.copy()
df_test_full['text'] = df_test_full.apply(lambda row: f"{row['location'] or ''} {row['keyword'] or ''} {row['text']}".strip(), axis=1)


### **Model Assessment**

In [8]:
def evaluate_classifier(y_true, y_pred):
    f1 = f1_score(y_true, y_pred, average='weighted')
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    cm = confusion_matrix(y_true, y_pred)
    
    metrics_df = pd.DataFrame({
        'Value': [f1, precision, recall]
    }, index=['F1 Score', 'Precision', 'Recall'])
    
    cm_df = pd.DataFrame(cm, columns=['Predicted Negative', 'Predicted Positive'], index=['Actual Negative', 'Actual Positive'])
    
    return metrics_df, cm_df

### **ROBERT Tokenizer**

##### _ROBERT Tokenizer + split validation_

In [9]:
robert_tokenizer_full = RobertaTokenizer.from_pretrained("roberta-base")

robert_encodings_full = robert_tokenizer_full(list(df_train_full['text']), truncation=True, padding=True, max_length=256)
robert_labels_full = torch.tensor(list(df_train_full['target']))

robert_input_ids_train_full, robert_input_ids_valid_full, \
robert_attention_mask_train_full, robert_attention_mask_valid_full, robert_y_train_full, robert_y_valid_full = train_test_split(
    robert_encodings_full['input_ids'], 
    robert_encodings_full['attention_mask'], 
    robert_labels_full, 
    test_size=0.15, 
    stratify=robert_labels_full, 
    random_state=0
)

robert_train_encodings_full = {
    'input_ids': torch.tensor(robert_input_ids_train_full),
    'attention_mask': torch.tensor(robert_attention_mask_train_full)
}

robert_valid_encodings_full = {
    'input_ids': torch.tensor(robert_input_ids_valid_full),
    'attention_mask': torch.tensor(robert_attention_mask_valid_full)
}

robert_train_dataset_full = Dataset.from_dict({
    "input_ids": robert_train_encodings_full['input_ids'],
    "attention_mask": robert_train_encodings_full['attention_mask'],
    "labels": robert_y_train_full
})

robert_valid_dataset_full = Dataset.from_dict({
    "input_ids": robert_valid_encodings_full['input_ids'],
    "attention_mask": robert_valid_encodings_full['attention_mask'],
    "labels": robert_y_valid_full
})

robert_test_encodings_full = robert_tokenizer_full(list(df_test_full['text']), truncation=True, padding=True, max_length=256)

robert_test_encodings_full = {
    key: torch.tensor(val) for key, val in robert_test_encodings_full.items()
}

robert_test_dataset_full = Dataset.from_dict({
    "input_ids": robert_test_encodings_full['input_ids'],
    "attention_mask": robert_test_encodings_full['attention_mask']
})

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

c:\Users\cauchepy\Git\.venv\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\cauchepy\.cache\huggingface\hub\models--roberta-base. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

##### _Save HuggingFace ROBERT Tokenizer_

In [10]:
robert_tokenizer_full.push_to_hub("yanncauchepin/kaggle_disastertweets_robert_tokenizer")

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

c:\Users\cauchepy\Git\.venv\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\cauchepy\.cache\huggingface\hub\models--yanncauchepin--kaggle_disastertweets_robert_tokenizer. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


CommitInfo(commit_url='https://huggingface.co/yanncauchepin/kaggle_disastertweets_robert_tokenizer/commit/954fa6db2bd7b154bf8af92c400da7e0e9808d2e', commit_message='Upload tokenizer', commit_description='', oid='954fa6db2bd7b154bf8af92c400da7e0e9808d2e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/yanncauchepin/kaggle_disastertweets_robert_tokenizer', endpoint='https://huggingface.co', repo_type='model', repo_id='yanncauchepin/kaggle_disastertweets_robert_tokenizer'), pr_revision=None, pr_num=None)

### **ROBERT Training from scratch**

##### _ROBERT Transformers + local save_

In [11]:
robert_model_full = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)

training_args = TrainingArguments(
    hub_model_id="kaggle_disastertweets_robert_model",
    output_dir="./robert_results",
    num_train_epochs=5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./robert_logs",
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none"
)

robert_trainer_full = Trainer(
    model=robert_model_full,
    args=training_args,
    train_dataset=robert_train_dataset_full,
    eval_dataset=robert_valid_dataset_full
)

robert_trainer_full.train()

robert_trainer_full.save_model("robert_model") # Local

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/8090 [00:00<?, ?it/s]

{'loss': 0.717, 'grad_norm': 3.1206634044647217, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.01}
{'loss': 0.7064, 'grad_norm': 2.3886795043945312, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.01}
{'loss': 0.7375, 'grad_norm': 5.718069553375244, 'learning_rate': 3e-06, 'epoch': 0.02}
{'loss': 0.6954, 'grad_norm': 3.3371169567108154, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.02}
{'loss': 0.7052, 'grad_norm': 2.7036194801330566, 'learning_rate': 5e-06, 'epoch': 0.03}
{'loss': 0.7053, 'grad_norm': 6.718144416809082, 'learning_rate': 6e-06, 'epoch': 0.04}
{'loss': 0.7026, 'grad_norm': 3.003601312637329, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.04}
{'loss': 0.6973, 'grad_norm': 2.743086814880371, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.05}
{'loss': 0.6736, 'grad_norm': 3.3397388458251953, 'learning_rate': 9e-06, 'epoch': 0.06}
{'loss': 0.6681, 'grad_norm': 16.601938247680664, 'learning_rate': 1e-05, 'epoch': 0.06}
{'loss': 0.6355, 'grad_norm': 7.7

  0%|          | 0/286 [00:00<?, ?it/s]

...

{'eval_loss': 0.5573562979698181, 'eval_runtime': 5.4691, 'eval_samples_per_second': 208.811, 'eval_steps_per_second': 52.294, 'epoch': 3.0}
{'loss': 0.4674, 'grad_norm': 2.3321189880371094, 'learning_rate': 2.127799736495389e-05, 'epoch': 3.0}
{'loss': 0.6711, 'grad_norm': 2.306440591812134, 'learning_rate': 2.1212121212121215e-05, 'epoch': 3.01}
{'loss': 0.4717, 'grad_norm': 4.854271411895752, 'learning_rate': 2.114624505928854e-05, 'epoch': 3.02}
{'loss': 0.669, 'grad_norm': 5.370188236236572, 'learning_rate': 2.1080368906455864e-05, 'epoch': 3.02}
{'loss': 0.3719, 'grad_norm': 1.6877837181091309, 'learning_rate': 2.101449275362319e-05, 'epoch': 3.03}
{'loss': 0.6337, 'grad_norm': 2.715369939804077, 'learning_rate': 2.0948616600790517e-05, 'epoch': 3.03}
{'loss': 0.5945, 'grad_norm': 1.4047602415084839, 'learning_rate': 2.088274044795784e-05, 'epoch': 3.04}
{'loss': 0.4844, 'grad_norm': 1.717625379562378, 'learning_rate': 2.0816864295125166e-05, 'epoch': 3.05}
{'loss': 0.5964, 'grad

  0%|          | 0/286 [00:00<?, ?it/s]

{'eval_loss': 0.551849901676178, 'eval_runtime': 5.4879, 'eval_samples_per_second': 208.093, 'eval_steps_per_second': 52.114, 'epoch': 4.0}
{'loss': 0.5973, 'grad_norm': 5.37338924407959, 'learning_rate': 1.0606060606060607e-05, 'epoch': 4.0}
{'loss': 0.4571, 'grad_norm': 3.4924166202545166, 'learning_rate': 1.0540184453227932e-05, 'epoch': 4.01}
{'loss': 0.6802, 'grad_norm': 1.1163924932479858, 'learning_rate': 1.0474308300395258e-05, 'epoch': 4.02}
{'loss': 0.5583, 'grad_norm': 3.3449275493621826, 'learning_rate': 1.0408432147562583e-05, 'epoch': 4.02}
{'loss': 0.6348, 'grad_norm': 3.3291192054748535, 'learning_rate': 1.0342555994729908e-05, 'epoch': 4.03}
{'loss': 0.3833, 'grad_norm': 1.3166664838790894, 'learning_rate': 1.0276679841897234e-05, 'epoch': 4.04}
{'loss': 0.7669, 'grad_norm': 1.0696091651916504, 'learning_rate': 1.0210803689064559e-05, 'epoch': 4.04}
{'loss': 0.7292, 'grad_norm': 3.34525203704834, 'learning_rate': 1.0144927536231885e-05, 'epoch': 4.05}
{'loss': 0.7119, 

  0%|          | 0/286 [00:00<?, ?it/s]

{'eval_loss': 0.5437395572662354, 'eval_runtime': 6.6832, 'eval_samples_per_second': 170.877, 'eval_steps_per_second': 42.794, 'epoch': 5.0}
{'train_runtime': 1023.1907, 'train_samples_per_second': 31.622, 'train_steps_per_second': 7.907, 'train_loss': 0.594742638160036, 'epoch': 5.0}


##### _Save HugggingFace trained ROBERT Transformers_ 

In [12]:
robert_trainer_full.push_to_hub()

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/yanncauchepin/kaggle_disastertweets_robert_model/commit/9e5bc92d94fa2f14227b0cc707011efe66b38042', commit_message='End of training', commit_description='', oid='9e5bc92d94fa2f14227b0cc707011efe66b38042', pr_url=None, repo_url=RepoUrl('https://huggingface.co/yanncauchepin/kaggle_disastertweets_robert_model', endpoint='https://huggingface.co', repo_type='model', repo_id='yanncauchepin/kaggle_disastertweets_robert_model'), pr_revision=None, pr_num=None)

### **ROBERT Loading from pretrained**

In [13]:
robert_model_full = RobertaForSequenceClassification.from_pretrained("robert_model")

training_args = TrainingArguments(
    hub_model_id="kaggle_disastertweets_robert_model",
    output_dir="./robert_results",
    num_train_epochs=5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./robert_logs",
    logging_steps=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none"
)

robert_trainer_full = Trainer(
    model=robert_model_full,
    args=training_args,
    train_dataset=robert_train_dataset_full,
    eval_dataset=robert_valid_dataset_full
)

### **ROBERT Assessment**

In [14]:
robert_predictions_full = robert_trainer_full.predict(robert_valid_dataset_full)
robert_logits_full = robert_predictions_full.predictions
robert_y_pred_full = np.argmax(robert_logits_full, axis=1)

robert_trainer_full_assessement = evaluate_classifier(robert_y_valid_full.numpy(), robert_y_pred_full)
display(robert_trainer_full_assessement[0])
display(robert_trainer_full_assessement[1])

  0%|          | 0/286 [00:00<?, ?it/s]

,Value
F1 Score,0.796008
Precision,0.799704
Recall,0.798599


,Predicted Negative,Predicted Positive
Actual Negative,571,80
Actual Positive,150,341


### **ROBERT Test Predictions**

##### _ROBERT Test Predictions_

In [15]:
robert_test_predictions_full = robert_trainer_full.predict(robert_test_dataset_full)
robert_test_logits_full = robert_test_predictions_full.predictions
robert_test_y_pred_full = np.argmax(robert_test_logits_full, axis=1)

robert_test_submission_full = pd.DataFrame({
    'id': df_test_full.index,
    'target': robert_test_y_pred_full.flatten()
})

  0%|          | 0/816 [00:00<?, ?it/s]

##### _Save HuggingFace ROBERT Test Predictions_

In [16]:
hf_robert_test_submission_full = Dataset.from_pandas(robert_test_submission_full)
hf_robert_test_submission_full.push_to_hub("yanncauchepin/kaggle_disastertweets_robert_submission_df")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/4 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/299 [00:00<?, ?B/s]

c:\Users\cauchepy\Git\.venv\lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\cauchepy\.cache\huggingface\hub\datasets--yanncauchepin--kaggle_disastertweets_robert_submission_df. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


CommitInfo(commit_url='https://huggingface.co/datasets/yanncauchepin/kaggle_disastertweets_robert_submission_df/commit/98f152e9b5a447184bcec4d0abf0a51ebc354c19', commit_message='Upload dataset', commit_description='', oid='98f152e9b5a447184bcec4d0abf0a51ebc354c19', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/yanncauchepin/kaggle_disastertweets_robert_submission_df', endpoint='https://huggingface.co', repo_type='dataset', repo_id='yanncauchepin/kaggle_disastertweets_robert_submission_df'), pr_revision=None, pr_num=None)